# Timer

In [ ]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['PriceChange', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'Trend': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 100


  9%|██████████████▌                                                                                                                                                   | 9/100 [00:25<02:55,  1.93s/it]

# Check local statistics

In [ ]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

In [5]:
# 15m/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,Trend_timeperiod,Trend_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
14,PriceChange_Trend,3,12,0,2.1507,9.1589,9.1589,9.1589,10.0000,10.0000,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,-0.90,-3.32,-2.17,-1.93,-3.79,-3.27,-2.63,-2.31,-2.94,-2.60,-2.31,-2.14,-2.36,-1.93,-2.28,-2.82,-2.20,-1.80,-1.88,-2.29,-2.21,-2.40,-2.06,-1.97,2,9.198358,-2.354583,16.396717,-4.709167
13,PriceChange_Trend,3,10,0,2.1507,9.1589,9.1589,9.1589,10.0000,10.0000,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,9.5074,-0.90,-3.32,-2.17,-1.93,-3.79,-3.27,-2.63,-2.31,-2.94,-2.60,-2.31,-2.14,-2.36,-1.93,-2.28,-2.82,-2.20,-1.80,-1.88,-2.29,-2.21,-2.40,-2.06,-1.97,2,9.198358,-2.354583,16.396717,-4.709167
50,PriceChange_Trend,11,4,0,1.3227,2.4615,2.6115,2.9298,2.8655,3.1707,3.0831,2.7546,2.7546,2.8093,2.8699,2.9520,3.0540,3.0945,3.0945,2.8997,2.3034,1.2110,1.1115,0.8560,0.7835,0.7767,0.7026,0.6396,-0.54,-0.82,-1.86,-1.67,-1.54,-1.62,-1.54,-1.54,-1.57,-1.62,-1.73,-1.35,-1.77,-1.89,-1.68,-1.26,-1.84,-1.59,-1.35,-0.93,-0.81,-0.48,-0.49,-0.39,11,2.213008,-1.328333,13.343092,-14.611667
55,PriceChange_Trend,12,4,0,1.3227,2.4615,2.6115,2.9298,2.8655,3.1707,3.0831,2.7546,2.7546,2.8093,2.8699,2.9520,3.0540,3.0945,3.0945,2.8997,2.3034,1.2110,1.1115,0.8560,0.7835,0.7767,0.7026,0.6396,-0.54,-0.82,-1.86,-1.67,-1.54,-1.62,-1.54,-1.54,-1.57,-1.62,-1.73,-1.35,-1.77,-1.89,-1.68,-1.26,-1.84,-1.59,-1.35,-0.93,-0.81,-0.48,-0.49,-0.39,11,2.213008,-1.328333,13.343092,-14.611667
56,PriceChange_Trend,12,6,0,1.3048,2.5071,2.6656,2.7911,2.7245,3.0561,2.9664,2.6273,2.6273,2.6860,2.7511,2.8027,2.8027,2.8461,2.8461,2.6548,2.0560,1.0555,0.9691,0.7430,0.6816,0.6756,0.6102,0.5547,-0.53,-1.33,-1.88,-1.46,-1.43,-1.62,-1.54,-1.42,-1.50,-1.41,-1.56,-1.20,-1.56,-1.81,-1.65,-1.17,-1.73,-1.26,-1.19,-0.88,-0.76,-0.41,0.16,-0.33,10,2.083558,-1.227917,10.835583,-12.279167
51,PriceChange_Trend,11,6,0,1.3048,2.5071,2.6656,2.7911,2.7245,3.0561,2.9664,2.6273,2.6273,2.6860,2.7511,2.8027,2.8027,2.8461,2.8461,2.6548,2.0560,1.0555,0.9691,0.7430,0.6816,0.6756,0.6102,0.5547,-0.53,-1.33,-1.88,-1.46,-1.43,-1.62,-1.54,-1.42,-1.50,-1.41,-1.56,-1.20,-1.56,-1.81,-1.65,-1.17,-1.73,-1.26,-1.19,-0.88,-0.76,-0.41,0.16,-0.33,10,2.083558,-1.227917,10.835583,-12.279167
29,PriceChange_Trend,6,12,0,0.9902,3.7682,4.4289,4.4289,5.1201,5.1201,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,3.9411,1.2719,-0.21,-1.97,-2.38,-1.45,-1.31,-1.48,-1.31,-1.38,-1.21,-1.38,-1.24,-1.48,-1.48,-1.45,-1.45,-1.45,-1.45,-1.31,-1.24,-1.17,-1.24,-1.31,-1.10,-1.31,3,4.437625,-1.365000,10.312875,-4.095000
18,PriceChange_Trend,4,10,0,0.9902,3.7682,4.4289,4.4289,5.1201,5.1201,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,3.9411,1.2719,-0.21,-1.97,-2.38,-1.45,-1.31,-1.48,-1.31,-1.38,-1.21,-1.38,-1.24,-1.48,-1.48,-1.45,-1.45,-1.45,-1.45,-1.31,-1.24,-1.17,-1.24,-1.31,-1.10,-1.31,3,4.437625,-1.365000,10.312875,-4.095000
24,PriceChange_Trend,5,12,0,0.9902,3.7682,4.4289,4.4289,5.1201,5.1201,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8396,4.8

# Save new config data to config file

In [10]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['PriceChange', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [20]}, 
              'Trend': {'timeperiod': [8], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'sell'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
20,72.930243,-1.306944,-879.743681,-157.711944,3071
19,73.131424,-1.325521,-881.025174,-153.987604,3038
17,73.016458,-1.380833,-906.773576,-147.793403,2972
18,72.718576,-1.329062,-908.998681,-150.019097,3005
16,73.201181,-1.421007,-909.151806,-143.919722,2933
15,73.250174,-1.485451,-924.197604,-142.200174,2911
12,74.196285,-1.676250,-927.744236,-130.194201,2805
13,73.203299,-1.593021,-954.058576,-131.837604,2836
11,73.823993,-1.782292,-954.826875,-126.921111,2770
